In [1]:
# Saab transformationv 2021.04.12
# modified from https://github.com/ChengyaoWang/PixelHop-_c-wSaab/blob/master/saab.py

import numpy as np
import numba
# from sklearn.decomposition import PCA, IncrementalPCA


@numba.jit(nopython = True, parallel = True)
def pca_cal(X: np.ndarray):
    cov = X.transpose() @ X
    eva, eve = np.linalg.eigh(cov)
    inds = eva.argsort()[::-1]
    eva = eva[inds]
    kernels = eve.transpose()[inds]
    return kernels, eva / (X.shape[0] - 1)

@numba.jit(forceobj = True, parallel = True)
def remove_mean(X: np.ndarray, feature_mean: np.ndarray):
    return X - feature_mean

@numba.jit(nopython = True, parallel = True)
def feat_transform(X: np.ndarray, kernel: np.ndarray):
    return X @ kernel.transpose()


class Saab():
    def __init__(self, num_kernels=-1, needBias=True, bias=0):
        self.num_kernels = num_kernels 
        self.needBias = needBias
        self.Bias_previous = bias # bias calculated from previous
        self.Bias_current = [] # bias for the current Hop
        self.Kernels = []
        self.Mean0 = [] # feature mean of AC
        self.Energy = [] # kernel energy list
        self.trained = False

    def fit(self, X): 
        assert (len(X.shape) == 2), "Input must be a 2D array!"
        X = X.astype('float32')
        
        # add bias from the previous Hop
        if self.needBias == True:
            X += self.Bias_previous
            
        # remove DC, get AC components
        dc = np.mean(X, axis = 1, keepdims = True)
        X = remove_mean(X, dc)
        
        # calcualte bias at the current Hop
        self.Bias_current = np.max(np.linalg.norm(X, axis=1))
        
        # remove feature mean --> self.Mean0
        self.Mean0 = np.mean(X, axis = 0, keepdims = True)
        X = remove_mean(X, self.Mean0)

        if self.num_kernels == -1:
            self.num_kernels = X.shape[-1]
        
        # Rewritten PCA Using Numpy
        kernels, eva = pca_cal(X)
        
        # Concatenate with DC kernel
        dc_kernel = 1 / np.sqrt(X.shape[-1]) * np.ones((1, X.shape[-1]))# / np.sqrt(largest_ev)
        kernels = np.concatenate((dc_kernel, kernels[:-1]), axis = 0)
        
        # Concatenate with DC energy
        largest_ev = np.var(dc * np.sqrt(X.shape[-1]))  
        energy = np.concatenate((np.array([largest_ev]), eva[:-1]), axis = 0)
        energy = energy / np.sum(energy)
        
        # store
        self.Kernels, self.Energy = kernels.astype('float32'), energy
        self.trained = True


    def transform(self, X):
        assert (self.trained == True), "Must call fit first!"
        X = X.astype('float32')
        
        # add bias from the previous Hop
        if self.needBias == True:
            X += self.Bias_previous
            
        # remove feature mean of AC
        X = remove_mean(X, self.Mean0)
        
        # convolve with DC and AC filters
        X = feat_transform(X, self.Kernels)
        
        return X

In [2]:
# load all images from covid-chestxray-dataset/images

import os
import cv2

# list all files in covid-chestxray-dataset/images
path = 'covid-chestxray-dataset/images'
files = os.listdir(path)

In [8]:
img = cv2.imread(os.path.join(path, files[0]), cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (420, 512))
img = img.reshape(1, 420, 512, 1)

print(img)
print(" input feature shape: %s"%str(img.shape))

[[[[ 38]
   [ 38]
   [ 38]
   ...
   [ 43]
   [ 43]
   [ 43]]

  [[ 45]
   [ 45]
   [ 44]
   ...
   [106]
   [104]
   [104]]

  [[101]
   [103]
   [101]
   ...
   [ 71]
   [ 71]
   [ 71]]

  ...

  [[115]
   [116]
   [115]
   ...
   [115]
   [116]
   [115]]

  [[115]
   [117]
   [115]
   ...
   [118]
   [117]
   [120]]

  [[119]
   [117]
   [119]
   ...
   [133]
   [133]
   [132]]]]
 input feature shape: (1, 420, 512, 1)


In [9]:
datasets = []
for i in range(len(files)):
    img = cv2.imread(os.path.join(path, files[i]), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (420, 512))
    img = img.reshape(1, 420, 512, 1)
    datasets.append(img)

datasets = np.concatenate(datasets, axis = 0)
print(" input feature shape: %s"%str(datasets.shape))

 input feature shape: (930, 420, 512, 1)


In [16]:
X = datasets.copy()
X = X.reshape(X.shape[0], -1)[0:50]
print(" input feature shape: %s"%str(X.shape))

 input feature shape: (50, 215040)


In [17]:
saab = Saab(num_kernels=-1, needBias=True, bias=0)
saab.fit(X)

MemoryError: Allocation failed (probably too large).